## imports

In [ ]:
import numpy as np
import pandas as pd

import os
from zipfile import ZipFile

In [ ]:
EXT_DATA = "..\data\external"
INT_DATA = "..\data\interim"

## functions

In [ ]:
def dir_digger(path, **args):
    files = args.get("files", [])
    ext = args.get("ext", "")
    for sub_path in os.listdir(path):
        new_path = os.path.join(path, sub_path)
        if os.path.isfile(new_path) & new_path.endswith(ext):
            files.append(new_path)
        elif os.path.isdir(new_path):
            files = dir_digger(new_path, files=files, ext=ext)
    return files   

In [ ]:
def read_csv_zip(path, **args):
    if path.endswith(".csv"):
        return pd.read_csv(path, **args)
    elif path.endswith(".csv.zip"):
        with ZipFile(path) as myzip:
            filename = os.path.splitext((os.path.basename(path)))[0]
            return pd.read_csv(myzip.open(filename), **args)

## parse files

### interim

In [ ]:
files = dir_digger(INT_DATA, ext=".parquet")
print(f"Found {len(files)} files:")
files

In [ ]:
for file in os.listdir(INT_DATA):
    print(f"\nFile {file}:")
    display(pd.read_parquet(os.path.join(INT_DATA, file)))

### external

In [ ]:
files = dir_digger(EXT_DATA, ext=".csv.zip")
print(f"Found {len(files)} files:")
files

In [ ]:
for file in files:
    print(f"\nFile {file}:")
    display(read_csv_zip(file))